Los datos provienen de un test de Autoconcepto que mide 5 componentes distintos. Además se han incluido algunas variables que se pueden usar como criterio. La lista con todas las variables está disponible en **listado de preguntas FINAL.xlsx**


El objetivo es ser capaz de predecir los ingresos mensuales (**A2**) utilizando la información de los componentes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

print(tf.__version__)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# funciones que nos van a ayudar

# normaliza un conjunto de datos
def norm(x, st):
    return((x - st['mean'])/st['std'])

# grafico de la historia    
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error')
  plt.plot(hist['epoch'], hist['loss'],'r--',
           label='Training Error')
  # plt.plot(hist['epoch'], hist['mse'],'b',
  #          label = 'mse')
  #plt.ylim([0,20])
  plt.legend()
  plt.show() # un €

In [ ]:
url = 'https://raw.githubusercontent.com/mcstllns/DeepLearning/main/datos%20para%20P07/Datos%20AF%20final.csv'
df = pd.read_csv(url)

In [ ]:
print(df.shape)
# print(df.head())
# print(df.columns)

In [ ]:
# Se preparan los datos para ser metidos en la red

X = df.iloc[:,np.r_[df.columns.get_loc("A01.10"):df.columns.get_loc("S04.14")+1, df.columns.get_loc("A2")]]
X = X.dropna()


# Esta parte separa en train y test, pero en nuestro caso no tiene sentido 
# porque el objetivo es descriptivo, no predictivo

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X.iloc[:,0:50], X.iloc[:,50], test_size=0.20)

x = X.iloc[:,0:50]
y = X.iloc[:,50]




In [ ]:
# Se normaliza x e y 

x_stats = x.describe().transpose()
y_stats = y.describe().transpose()

x_norm = norm(x, x_stats)
y_norm = norm(y, y_stats)

In [ ]:
print(x_norm.shape)
print(y_norm.shape)


In [ ]:
# Construcción del modelo

model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape=[( len(x.keys()) )]))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

model.summary()

model.compile(loss='mse',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['mse','mae'])

In [ ]:
# Lo que tiene más sentido es normalizar x pero utilizar y en su escala original

history = model.fit(x_norm, y,
                    epochs = 500,
                    batch_size = 25,
                    verbose = 1) 

In [ ]:
plot_history(history)

In [ ]:
model.evaluate(x_norm, y)

In [ ]:
# Como las variables son cuantitativas un grafico de dispersion podría estar perfecto

yp = model.predict(x_norm)


df = pd.DataFrame({'y': y, 'yp': yp.flatten()})
df.plot.scatter(x='y', y='yp')
plt.xlim([0,5000])
plt.ylim([0,5000])
plt.show()

# Tarea

Utilice una arquitectura funcional para separar los componentes en líneas de procesamiento. Interprete los resultados.

Utilice los datos para predecir otras variables relevantes del fichero.
